In [1]:
import spacy
spacy.require_gpu()

True

In [24]:
model2run = {    
    'ggponc2_fine_long_bigbio_kb' : {
        'deepset/gbert-base' : '2024-01-02_09-33-47-cuda-5'
    },
    'ggponc2_fine_short_bigbio_kb' : {
        'deepset/gbert-base' : ...
    }
}

In [35]:
config = 'ggponc2_fine_short_bigbio_kb'

In [10]:

checkpoint = 'deepset/gbert-base'

# ner_model = spacy.load('de_ggponc_medbertde')

## or
from spacy_ner import chunk_and_ngram_suggester
run = model2run[config][checkpoint]
ner_model = spacy.load(f'spacy_ner/training/{run}/model-best')

In [40]:
from spacy.tokens import DocBin
from itertools import islice
from xmen.data import from_spacy

spacy_test_db = DocBin(store_user_data=True).from_disk(f'data/{config}/ggponc_spacy/test.spacy')
spacy_test = list(spacy_test_db.get_docs(ner_model.vocab))

In [41]:
ents = [e for s in spacy_test for e in s.spans['entities']]

In [42]:
len(ents)

171358

In [12]:
spacy_test_ds = from_spacy(spacy_test, 'entities')

In [13]:
from tqdm.auto import tqdm
test_data = [d.text for d in spacy_test]
pred = list(tqdm(ner_model.pipe(test_data,  batch_size=128), total=len(test_data)))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 13714/13714 [02:51<00:00, 79.98it/s]


In [14]:
pred_ds = from_spacy(pred, 'entities')

In [15]:
def filter_entities(semantic_class):
    def _filter(d):
        res = []
        for e in d['entities']:
            if e["type"] == semantic_class:
                res.append(e)
        return {"entities" : res}
    return _filter

In [19]:
semantic_classes = ['Diagnosis_or_Pathology', 'Other_Finding', 'Clinical_Drug', 'Nutrient_or_Body_Substance', 'Therapeutic', 'Diagnostic', 'External_Substance']

In [17]:
from xmen.evaluation import evaluate, error_analysis
import pandas as pd

res = []
for s in semantic_classes:
    e = evaluate(spacy_test_ds.map(filter_entities(s)), pred_ds.map(filter_entities(s)), ner_only=True, metrics=['strict'])['strict']
    e['tag'] = s
    res.append(e)
res = pd.DataFrame(res)

Map: 100%|███████████████████████████████████████████████████████████████████████████████████| 13714/13714 [00:01<00:00, 7671.46 examples/s]


In [18]:
res.set_index('tag')

,precision,recall,fscore,ptp,fp,rtp,fn,n_docs_system,n_annos_system,n_docs_gold,n_annos_gold
tag,,,,,,,,,,,
Diagnosis_or_Pathology,0.766281,0.779034,0.772605,8472,2584,8472,2403,13714,11073,13714,10883
Other_Finding,0.590572,0.610040,0.600148,4460,3092,4460,2851,13714,7571,13714,7316
Clinical_Drug,0.790004,0.798668,0.794312,2039,542,2039,514,13714,2581,13714,2555
Nutrient_or_Body_Substance,0.523757,0.738318,0.612799,474,431,474,168,13714,905,13714,643
Therapeutic,0.768346,0.777460,0.772876,7078,2134,7078,2026,13714,9227,13714,9110
External_Substance,0.515152,0.393064,0.445902,68,64,68,105,13714,132,13714,173
Diagnostic,0.655497,0.729150,0.690364,2719,1429,2719,1010,13714,4156,13714,3734
